In [ ]:
import os
import json
from tqdm import tqdm
from collections import defaultdict

# 분류 매핑
CATEGORY_MAPPING = {
    "교통(자동차)": ["교통", "도로교통", "자동차", "음주운전", "무면허운전", "사고후미조치"],
    "사기 및 형사": ["사기", "절도", "횡령", "배임", "상해", "폭행", "성범죄", "마약", "공무집행방해", "장물", "폭력"],
    "부동산": ["건물명도", "임대차보증금", "소유권", "토지", "부동산", "전세", "분양", "점포", "양도", "철거"],
    "노동": ["근로", "최저임금", "퇴직금", "해고", "산재", "임금", "노임"],
    "대금": ["대금", "공사대금", "물품대금", "매매대금", "약정금", "지급", "초과지급", "연대채무금"],
    "손해배상": ["손해배상", "배상", "위자료", "상해배상", "피해배상"],
    "상속": ["상속", "유류분", "유언", "상속재산", "상속회복", "유산분할", "상속등기", "상속포기"],
    "계약": ["계약", "계약금", "계약해지", "계약해제", "계약위반", "계약불이행", "청약", "승낙", "해약"]
}


CATEGORIES = list(CATEGORY_MAPPING.keys()) + ["기타"]

def get_categories_for_casenames(casenames, mapping=CATEGORY_MAPPING):
    casename_list = [c.strip() for c in casenames.split(",")]
    result = set()
    for casename in casename_list:
        for cat, keywords in mapping.items():
            if any(k in casename for k in keywords):
                result.add(cat)
    if not result:
        result.add("기타")
    return list(result)


In [6]:
ROOT_DIR = "판결문"
OUT_DIR = "precomputed"
os.makedirs(OUT_DIR, exist_ok=True)

# 분류별 저장할 리스트
classified_documents = defaultdict(list)   # content(str) 저장
classified_metadatas = defaultdict(list)   # dict 저장

for subdir in ["1", "2", "3", "4"]:
    sub_path = os.path.join(ROOT_DIR, subdir)
    for fname in tqdm(os.listdir(sub_path), desc=f"Processing {subdir}"):
        if not fname.endswith(".json"):
            continue
        fpath = os.path.join(sub_path, fname)
        with open(fpath, encoding="utf-8") as f:
            data = json.load(f)
        casenames = data.get("casenames", "")
        categories = get_categories_for_casenames(casenames)
        content = (
            f"사건명: {data.get('casenames', '')}\n"
            + "\n".join(data.get("sentences", []))
        )
        meta = {
            "source": fname,
            "type": "json",
            "casenames": data.get("casenames", ""),
            "announce_date": data.get("announce_date", "")
        }
        for cat in categories:
            classified_documents[cat].append(content)
            classified_metadatas[cat].append(meta)


Processing 4: 100%|██████████| 19027/19027 [00:02<00:00, 7740.05it/s]


In [7]:
print("\n=== 카테고리별 JSON 파일 개수 ===")
for cat in classified_documents.keys():
    print(f"{cat}: {len(classified_documents[cat])}개")


=== 카테고리별 JSON 파일 개수 ===
대금: 14498개
기타: 23546개
손해배상: 32288개
부동산: 5431개
교통(자동차): 1178개
노동: 2836개
사기 및 형사: 479개
계약: 249개
상속: 15개


In [8]:
from tqdm import tqdm
from langchain_openai import OpenAIEmbeddings
import pickle
import os

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

for cat in CATEGORIES:
    if not classified_documents[cat]:
        continue
    docs = classified_documents[cat]
    vectors = []
    # tqdm으로 카테고리 내부 문서별 임베딩 진행률 출력
    for doc in tqdm(docs, desc=f"{cat} 임베딩", leave=True):
        vectors.append(embeddings.embed_query(doc))
    # 파일 저장
    with open(os.path.join(OUT_DIR, f"{cat}_documents.pkl"), "wb") as f:
        pickle.dump(docs, f)
    with open(os.path.join(OUT_DIR, f"{cat}_embeddings.pkl"), "wb") as f:
        pickle.dump(vectors, f)



기타 임베딩: 100%|██████████| 23546/23546 [3:59:21<00:00,  1.64it/s]     
